# Train the model

- Train the model on your own dataset
- use Efficientnet-b5 as backbone
- And Imagenet as weight(you may use noisy-student also)
- Train on Tpu for 40 epocs
- Use Gridmasking and Mixup technique
- Also may apply cutout/cutmix/Augmix/mixup only/etc...

In [1]:
!pip install ../input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl

Processing /kaggle/input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl


In [2]:
import os
import numpy as np
import tensorflow as tf
import math

def transform(image, inv_mat, image_shape):
    h, w, c = image_shape
    cx, cy = w//2, h//2
    new_xs = tf.repeat( tf.range(-cx, cx, 1), h)
    new_ys = tf.tile( tf.range(-cy, cy, 1), [w])
    new_zs = tf.ones([h*w], dtype=tf.int32)
    old_coords = tf.matmul(inv_mat, tf.cast(tf.stack([new_xs, new_ys, new_zs]), tf.float32))
    old_coords_x, old_coords_y = tf.round(old_coords[0, :] + w//2), tf.round(old_coords[1, :] + h//2)
    clip_mask_x = tf.logical_or(old_coords_x<0, old_coords_x>w-1)
    clip_mask_y = tf.logical_or(old_coords_y<0, old_coords_y>h-1)
    clip_mask = tf.logical_or(clip_mask_x, clip_mask_y)
    old_coords_x = tf.boolean_mask(old_coords_x, tf.logical_not(clip_mask))
    old_coords_y = tf.boolean_mask(old_coords_y, tf.logical_not(clip_mask))
    new_coords_x = tf.boolean_mask(new_xs+cx, tf.logical_not(clip_mask))
    new_coords_y = tf.boolean_mask(new_ys+cy, tf.logical_not(clip_mask))
    old_coords = tf.cast(tf.stack([old_coords_y, old_coords_x]), tf.int32)
    new_coords = tf.cast(tf.stack([new_coords_y, new_coords_x]), tf.int64)
    rotated_image_values = tf.gather_nd(image, tf.transpose(old_coords))
    rotated_image_channel = list()
    for i in range(c):
        vals = rotated_image_values[:,i]
        sparse_channel = tf.SparseTensor(tf.transpose(new_coords), vals, [h, w])
        rotated_image_channel.append(tf.sparse.to_dense(sparse_channel, default_value=0, validate_indices=False))
    return tf.transpose(tf.stack(rotated_image_channel), [1,2,0])

def random_rotate(image, angle, image_shape):
    def get_rotation_mat_inv(angle):
        # transform to radian
        angle = math.pi * angle / 180
        cos_val = tf.math.cos(angle)
        sin_val = tf.math.sin(angle)
        one = tf.constant([1], tf.float32)
        zero = tf.constant([0], tf.float32)
        rot_mat_inv = tf.concat([cos_val, sin_val, zero, -sin_val, cos_val, zero, zero, zero, one], axis=0)
        rot_mat_inv = tf.reshape(rot_mat_inv, [3,3])
        return rot_mat_inv
    angle = float(angle) * tf.random.normal([1],dtype='float32')
    rot_mat_inv = get_rotation_mat_inv(angle)
    return transform(image, rot_mat_inv, image_shape)


def GridMask(image_height, image_width, d1, d2, rotate_angle=1, ratio=0.5):
    h, w = image_height, image_width
    hh = int(np.ceil(np.sqrt(h*h+w*w)))
    hh = hh+1 if hh%2==1 else hh
    d = tf.random.uniform(shape=[], minval=d1, maxval=d2, dtype=tf.int32)
    l = tf.cast(tf.cast(d,tf.float32)*ratio+0.5, tf.int32)

    st_h = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)
    st_w = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)

    y_ranges = tf.range(-1 * d + st_h, -1 * d + st_h + l)
    x_ranges = tf.range(-1 * d + st_w, -1 * d + st_w + l)

    for i in range(0, hh//d+1):
        s1 = i * d + st_h
        s2 = i * d + st_w
        y_ranges = tf.concat([y_ranges, tf.range(s1,s1+l)], axis=0)
        x_ranges = tf.concat([x_ranges, tf.range(s2,s2+l)], axis=0)

    x_clip_mask = tf.logical_or(x_ranges < 0 , x_ranges > hh-1)
    y_clip_mask = tf.logical_or(y_ranges < 0 , y_ranges > hh-1)
    clip_mask = tf.logical_or(x_clip_mask, y_clip_mask)

    x_ranges = tf.boolean_mask(x_ranges, tf.logical_not(clip_mask))
    y_ranges = tf.boolean_mask(y_ranges, tf.logical_not(clip_mask))

    hh_ranges = tf.tile(tf.range(0,hh), [tf.cast(tf.reduce_sum(tf.ones_like(x_ranges)), tf.int32)])
    x_ranges = tf.repeat(x_ranges, hh)
    y_ranges = tf.repeat(y_ranges, hh)

    y_hh_indices = tf.transpose(tf.stack([y_ranges, hh_ranges]))
    x_hh_indices = tf.transpose(tf.stack([hh_ranges, x_ranges]))

    y_mask_sparse = tf.SparseTensor(tf.cast(y_hh_indices, tf.int64),  tf.zeros_like(y_ranges), [hh, hh])
    y_mask = tf.sparse.to_dense(y_mask_sparse, 1, False)

    x_mask_sparse = tf.SparseTensor(tf.cast(x_hh_indices, tf.int64), tf.zeros_like(x_ranges), [hh, hh])
    x_mask = tf.sparse.to_dense(x_mask_sparse, 1, False)

    mask = tf.expand_dims( tf.clip_by_value(x_mask + y_mask, 0, 1), axis=-1)

    mask = random_rotate(mask, rotate_angle, [hh, hh, 1])
    mask = tf.image.crop_to_bounding_box(mask, (hh-h)//2, (hh-w)//2, image_height, image_width)

    return mask

def apply_grid_mask(image, image_shape):
    AugParams = {
        'd1' : 100,
        'd2': 160,
        'rotate' : 45,
        'ratio' : 0.3
    }
    mask = GridMask(image_shape[0], image_shape[1], AugParams['d1'], AugParams['d2'], AugParams['rotate'], AugParams['ratio'])
    if image_shape[-1] == 3:
        mask = tf.concat([mask, mask, mask], axis=-1)
    return image * tf.cast(mask, tf.float32)


def gridmask(img_batch, label_batch, batch_size):
    return apply_grid_mask(img_batch, (160, 256, 3)), label_batch

In [3]:
import os
import numpy as np
import pandas as pd
import argparse
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from kaggle_datasets import KaggleDatasets

from tensorflow.keras import layers as L
import efficientnet.tfkeras as efn


def normalize(image):
  # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/main.py#L325-L326
  # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/efficientnet_builder.py#L31-L32
  image -= tf.constant([0.485 * 255, 0.456 * 255, 0.406 * 255])  # RGB
  image /= tf.constant([0.229 * 255, 0.224 * 255, 0.225 * 255])  # RGB
  return image


def get_model(input_size, backbone='efficientnet-b0', weights='imagenet', tta=False):
  print(f'Using backbone {backbone} and weights {weights}')
  x = L.Input(shape=input_size, name='imgs', dtype='float32')
  y = normalize(x)
  if backbone.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

  y = model_fn(input_shape=input_size, weights=weights, include_top=False)(y)
  y = L.GlobalAveragePooling2D()(y)
  y = L.Dropout(0.2)(y)
  # 1292 of 1295 are present
  y = L.Dense(1292, activation='softmax')(y)
  model = tf.keras.Model(x, y)

  if tta:
    assert False, 'This does not make sense yet'
    x_flip = tf.reverse(x, [2])  # 'NHWC'
    y_tta = tf.add(model(x), model(x_flip)) / 2.0
    tta_model = tf.keras.Model(x, y_tta)
    return model, tta_model

  return model

#Applying the Mixup
def mixup(img_batch, label_batch, batch_size):
  # https://github.com/tensorpack/tensorpack/blob/master/examples/ResNet/cifar10-preact18-mixup.py
  weight = tf.random.uniform([batch_size])
  x_weight = tf.reshape(weight, [batch_size, 1, 1, 1])
  y_weight = tf.reshape(weight, [batch_size, 1])
  index = tf.random.shuffle(tf.range(batch_size, dtype=tf.int32))
  x1, x2 = img_batch, tf.gather(img_batch, index)
  img_batch = x1 * x_weight + x2 * (1. - x_weight)
  y1, y2 = label_batch, tf.gather(label_batch, index)
  label_batch = y1 * y_weight + y2 * (1. - y_weight)
  return img_batch, label_batch


def get_strategy():
  # Detect hardware, return appropriate distribution strategy
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  except ValueError:
    tpu = None

  if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
  else:
    strategy = tf.distribute.get_strategy()

  print('REPLICAS: ', strategy.num_replicas_in_sync)
  return strategy


def one_hot(image, label):
  label = tf.one_hot(label, 1292)
  return image, label


def read_tfrecords(example, input_size):
  features = {
      'img': tf.io.FixedLenFeature([], tf.string),
      'image_id': tf.io.FixedLenFeature([], tf.int64),
      'grapheme_root': tf.io.FixedLenFeature([], tf.int64),
      'vowel_diacritic': tf.io.FixedLenFeature([], tf.int64),
      'consonant_diacritic': tf.io.FixedLenFeature([], tf.int64),
      'unique_tuple': tf.io.FixedLenFeature([], tf.int64),
  }
  example = tf.io.parse_single_example(example, features)
  img = tf.image.decode_image(example['img'])
  img = tf.reshape(img, input_size + (1, ))
  img = tf.cast(img, tf.float32)
  # grayscale -> RGB
  img = tf.repeat(img, 3, -1)

  # image_id = tf.cast(example['image_id'], tf.int32)
  # grapheme_root = tf.cast(example['grapheme_root'], tf.int32)
  # vowel_diacritic = tf.cast(example['vowel_diacritic'], tf.int32)
  # consonant_diacritic = tf.cast(example['consonant_diacritic'], tf.int32)
  unique_tuple = tf.cast(example['unique_tuple'], tf.int32)
  return img, unique_tuple


def main():
  parser = argparse.ArgumentParser()
  parser.add_argument('--model_id', type=int, default=0)
  parser.add_argument('--seed', type=int, default=123)
  parser.add_argument('--lr', type=float, default=1e-4)
  parser.add_argument('--input_size', type=str, default='160,256')
  parser.add_argument('--batch_size', type=int, default=128)
  parser.add_argument('--epochs', type=int, default=40)
  parser.add_argument('--backbone', type=str, default='efficientnet-b5')
  parser.add_argument('--weights', type=str, default='imagenet')
  args, _ = parser.parse_known_args()

  args.input_size = tuple(int(x) for x in args.input_size.split(','))
  np.random.seed(args.seed)
  tf.random.set_seed(args.seed)

  # build the model
  strategy = get_strategy()
  with strategy.scope():
    model = get_model(input_size=args.input_size + (3, ), backbone=args.backbone,
        weights=args.weights)

  model.compile(optimizer=Adam(lr=args.lr, beta_1=0.9, beta_2=0.999, decay=0.00001),
                loss=categorical_crossentropy,
                metrics=[categorical_accuracy, top_k_categorical_accuracy])
  # print(model.summary())

  # create the training and validation datasets
  ds_path = KaggleDatasets().get_gcs_path('bengali-tfrecords-v010')
  train_fns = tf.io.gfile.glob(os.path.join(ds_path, 'records/train*.tfrec'))
  train_ds = tf.data.TFRecordDataset(train_fns)
  train_ds = train_ds.map(lambda e: read_tfrecords(e, args.input_size))
  train_ds = train_ds.repeat().batch(args.batch_size)
  train_ds = train_ds.map(one_hot)
  train_ds = train_ds.map(lambda a,b: gridmask(a,b, args.batch_size))
  train_ds = train_ds.map(lambda a, b: mixup(a, b, args.batch_size))

  val_fns = tf.io.gfile.glob(os.path.join(ds_path, 'records/val*.tfrec'))
  val_ds = tf.data.TFRecordDataset(val_fns)
  val_ds = val_ds.map(lambda e: read_tfrecords(e, args.input_size))
  val_ds = val_ds.batch(args.batch_size)
  val_ds = val_ds.map(one_hot)

  # train
  num_train_samples = sum(int(fn.split('_')[2]) for fn in train_fns)
  # num_val_samples = sum(int(fn.split('_')[2]) for fn in val_fns)
  steps_per_epoch = num_train_samples // args.batch_size
  print(f'Training on {num_train_samples} samples. Each epochs requires {steps_per_epoch} steps')
  h = model.fit(train_ds, steps_per_epoch=steps_per_epoch, epochs=args.epochs, verbose=1,
      validation_data=val_ds)
  print(h)
  weight_fn = 'model-%04d.h5' % args.model_id
  model.save_weights(weight_fn)
  print(f'Saved weights to: {weight_fn}')


main()


Running on TPU  ['10.0.0.2:8470']
REPLICAS:  8
Using backbone efficientnet-b5 and weights imagenet
115515392/115515256 [==============================] - 4s 0us/step
Training on 160672 samples. Each epochs requires 1255 steps
Train for 1255 steps
Epoch 1/40
1255/1255 [==============================] - 422s 336ms/step - loss: 6.0900 - categorical_accuracy: 0.0815 - top_k_categorical_accuracy: 0.2051 - val_loss: 1.9880 - val_categorical_accuracy: 0.5762 - val_top_k_categorical_accuracy: 0.8819
Epoch 2/40
1255/1255 [==============================] - 257s 205ms/step - loss: 3.7096 - categorical_accuracy: 0.5150 - top_k_categorical_accuracy: 0.7798 - val_loss: 0.5185 - val_categorical_accuracy: 0.8948 - val_top_k_categorical_accuracy: 0.9847
Epoch 3/40
1255/1255 [==============================] - 257s 205ms/step - loss: 2.9910 - categorical_accuracy: 0.6799 - top_k_categorical_accuracy: 0.8726 - val_loss: 0.3178 - val_categorical_accuracy: 0.9349 - val_top_k_categorical_accuracy: 0.9926
Epo

In [4]:
ls

__notebook__.ipynb  model-0000.h5
